# Projekt zaliczeniowy na przedmiot Eksploracja i Wizualizacja Danych
Autor: Michał Brodacki, s32038
## Cel projektu
Celem projektu jest rozpoznanie na podstawie danych pozycyjnych koszykarza oddającego rzut, czy będzie on liczony jako $3$--punktowy, czy jako $2$--punktowy.
## Dane
Do projektu wykorzystam dane NBA 2023 Player Shot Dataset dostępne pod linkiem: 
https://www.kaggle.com/datasets/dhavalrupapara/nba-2023-player-shot-dataset/?select=2_james_harden_shot_chart_2023.csv

Zawierają one informacje na temat sytuacji rzutowych trzech koszykarzy występujących akutalnie w NBA: Stephena Curry'ego, Jamesa Hardena oraz Lebrona Jamesa. Szczegółowe informacje dotyczące każdej ze zmiennych można znaleźć pod powyższym linkiem

In [61]:
import pandas as pd #wstępna obróbka danych

import matplotlib.pyplot as plt # do wykresów

In [62]:
Lebron = pd.read_csv("1_lebron_james_shot_chart_1_2023.csv")
Harden = pd.read_csv("2_james_harden_shot_chart_2023.csv")
Curry = pd.read_csv("3_stephen_curry_shot_chart_2023.csv")
Lebron

,top,left,date,qtr,time_remaining,result,shot_type,distance_ft,lead,lebron_team_score,opponent_team_score,opponent,team,season,color
0,310,203,"Oct 18, 2022",1st Qtr,09:26,False,3,26,False,2,2,GSW,LAL,2023,red
1,213,259,"Oct 18, 2022",1st Qtr,08:38,False,2,16,False,4,5,GSW,LAL,2023,red
2,143,171,"Oct 18, 2022",1st Qtr,08:10,False,2,11,False,4,7,GSW,LAL,2023,red
3,68,215,"Oct 18, 2022",1st Qtr,05:24,True,2,3,False,12,19,GSW,LAL,2023,green
4,66,470,"Oct 18, 2022",1st Qtr,01:02,False,3,23,False,22,23,GSW,LAL,2023,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,118,274,"May 22, 2023",4th Qtr,06:49,False,2,8,False,98,102,DEN,LAL,2023,red
1529,109,221,"May 22, 2023",4th Qtr,05:03,False,2,6,False,100,102,DEN,LAL,2023,red
1530,78,254,"May 22, 2023",4th Qtr,03:20,True,2,3,False,104,107,DEN,LAL,2023,green
1531,78,46,"May 22, 2023",4th Qtr,00:26,False,2,20,False,111,113,DEN,LAL,2023,red


Widzimy, że zbiory zawierają 15 kolumn, szczegółowe opisy można znaleźć pod powyższym linkiem. 

## Wstępna ocena danych
Zmienną estymowaną w naszym przypadku będzie zmienna "shot_type", która mówi nam czy rzut został oddany za 2 punkty czy też za 3. W koszykówce istnieje jeszcze rzut za "1" punkt, sprawdźmy, czy też znajdziemy informacje o nim w tych datasetach.

In [63]:
print(Lebron['shot_type'].unique())
print(Curry['shot_type'].unique())
print(Harden['shot_type'].unique())

[3 2]
[2 3]
[3 2]


Takiej informacji nie znajdziemy, więc możemy śmiało go pominąć w naszym procesie myślowym.
Do moje predykcji będą nam potrzebne zmienne mówiące o pozycji zawodnika na boisku, oraz zmienna klasyfikująca, usuńmy resztę

In [64]:
Lebron = Lebron[['top', 'left', 'shot_type']]
Harden = Harden[['top', 'left', 'shot_type']]
Curry = Curry[['top', 'left', 'shot_type']]
Lebron

,top,left,shot_type
0,310,203,3
1,213,259,2
2,143,171,2
3,68,215,2
4,66,470,3
...,...,...,...
1528,118,274,2
1529,109,221,2
1530,78,254,2
1531,78,46,2


Klasyfikacja wydaje się możliwa, mamy dwie zmienne numeryczne odpowiadające za pozycję zawodnika oddajacego rzut w przestrzeni, oraz jedną klasyfikującą, która ma dwie wartości

## Przogotowanie Danych
Sprawdźmy czy mamy brakujące wartości w którymś z datasetów:

In [65]:
Lebron[Lebron.isna().any(axis=1) == True]

,top,left,shot_type


In [66]:
Harden[Harden.isna().any(axis=1) == True]

,top,left,shot_type


In [67]:
Curry[Curry.isna().any(axis=1) == True]

,top,left,shot_type


Nie mamy brakujących wartośc, sprawdźmy jeszcze w jaki sposób zakodowana jest zmienna, którą chcemy estymować:

In [68]:
Curry["shot_type"].dtype

dtype('int64')

Zmienna jest zakodowana jako INT, musimy zmienić ją na zmienną kategoryczną

In [69]:
Curry.loc[:, "shot_type"] = (Curry['shot_type'] == 3)
Lebron.loc[:, "shot_type"] = (Lebron['shot_type'] == 3)
Harden.loc[:, "shot_type"] = (Harden['shot_type'] == 3)
Curry.rename(columns={'shot_type' : 'is_three'}, inplace=True)
Lebron.rename(columns={'shot_type' : 'is_three'}, inplace=True)
Harden.rename(columns={'shot_type' : 'is_three'}, inplace=True)
Curry

C:\Users\micha\AppData\Local\Temp\ipykernel_16952\3482511981.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lebron.rename(columns={'shot_type' : 'is_three'}, inplace=True)


,top,left,is_three
0,63,300,False
1,133,389,False
2,326,247,True
3,249,89,True
4,282,158,True
...,...,...,...
1429,185,79,False
1430,193,19,True
1431,332,214,True
1432,310,240,True


Stworzyliśmy zmienną kategoryczną is_three, która mówi, czy oddany rzut, był trzypunktowy
## Modelowanie
- Modelowanie wykonam dla jednego z trzech zbiorów danych oraz dla wszystkich razem, ocenimy w ten sposób czy modele radzą sobie na mniejszym zbiorze, a może potrzebują dużego
- Użyte zostaną modele: KNN, Drzewa losowe (wersja klasyfikująca), #TODO more

In [71]:
df_all = pd.concat([Curry,Harden, Lebron], ignore_index = True)
df_all

,top,left,is_three
0,63,300,False
1,133,389,False
2,326,247,True
3,249,89,True
4,282,158,True
...,...,...,...
3987,118,274,False
3988,109,221,False
3989,78,254,False
3990,78,46,False


- Dla jednego zawodnika
    - KNN

-
    - Drzewo Losowe

- Dla wszystkich zawodników
    - KNN

-
    - Drzewo Losowe

## Ewaluacja Modeli

## Wnioski